In [24]:
class Card:
    def __init__(self, number:int, colour:str) -> None:
        self.number = number
        self.colour = colour
        self.wager = self._is_wager(self.number)

    def _is_wager(self,number):
        if number == 0:
            return True
    
    def __str__(self):
        return f"{self.colour} {self.number}"

In [2]:
import random
class Deck():
    def __init__(self, cards:list) -> None:
        self.list = [] 
        self.populate_deck(cards)
        self.len = len(cards)

    def populate_deck(self, cards:list):
        for card in cards:
            self.list.append(card)
    
    def shuffle_deck(self):
        random.shuffle(self.list )

In [3]:
class Player():
    def __init__(self, number:int):
        self.number = number
        self.hand = []
        self.expedition = {}

In [89]:
class GameManager():
    def __init__(self) -> None:
        self.colours = ["red","blue", "yellow", "green", "white"]
        self.player1= Player(1)
        self.player2= Player(2)
        self.players = {1:self.player1, 2:self.player2}
        self.player_turn = 1
        self.deck = self.generate_deck()
        self.discard_pile = self.generate_discard()
        self.score_dict = {}

    def next_player(self):
        if self.player_turn == 1:
            self.player_turn = 2
        else: 
            self.player_turn=1

    def generate_deck(self):
        numbers = [0,0,0,2,3,4,5,6,7,8,9,10]
        cards = []
        for colour in self.colours:
            for number in numbers:
                cards.append(Card(number, colour))
        deck = Deck(cards)
        return deck

    def generate_discard(self):
        discard_dict = {}
        for colour in self.colours:
            discard_dict[colour] = []
        return discard_dict

    def check_card_to_expedition(self,player_n:int, card_n:int):
        ''' check if card is too small to be played to the expedition'''
        player = self.players[player_n]
        card = player.hand[card_n]
        colour = card.colour

        if len(player.expedition[colour]) == 0:
            return True

        if player.expedition[colour][-1].number <= card.number:
            return True

        else:
            return False
 
    def is_move_legal(self,player:int, card:int, to_expedition:bool, draw_from:int):
        ''' Check if player is allowed to do this move'''
    	#is allowed to play that expedition
        if to_expedition:
            allowed_to_expedition =  self.check_card_to_expedition(player, card)
            if not allowed_to_expedition:
                return False
        
        #is allowed to drawi from pile
        if draw_from > -1:
            colour = self.colours[draw_from]
            if len (self.discard_pile[colour]) ==0:
                return False
        
        return True

    def take_random_action(self,player:int):
        ''' generate_random action'''
        card = random.randint(0,7)
        to_expedition = random.choice([True,False])
        draw_from = random.randint(-1,4)

        legal = self.is_move_legal(player,card, to_expedition, draw_from)
        if legal:
            self.take_action(player,card, to_expedition, draw_from)
            self.next_player()
        else: 
            self.take_random_action(player)

    def take_action(self, player_n:int, card_n:int, to_expedition:bool, draw_from:int ):
        ''' Player takes action, choosing card from hand (number 0 to 7), 
        chooses if they play it to expedition or to discard pile. Also chooses if they draw from  '''
        #print( f"Player {player_n} taking action on card n {card_n}, to expedition: {to_expedition}, draw_from: {draw_from}")
        player = self.players[player_n]
        card = player.hand.pop(card_n)
        colour = card.colour
        if to_expedition:
            player.expedition[colour].append(card)
        else:
            self.discard_pile[colour].append(card)

        if draw_from == -1:
            self.draw_card_from_deck(player)
        else:
            draw_colour = self.colours[draw_from]
            card = self.discard_pile[draw_colour].pop() #takes last item
            player.hand.append(card)

    def initiate_expeditions(self):
        #empty dicts for player expeditions
        for n,player in self.players.items():
            expedition_dict = {}
            for colour in self.colours:
                expedition_dict[colour] = []
            player.expedition= expedition_dict
    
    def draw_card_from_deck(self,player:Player):
        ''' draw a card from the deck and deal it to the player'''
        card = self.deck.list.pop()
        player.hand.append(card)

    def deal_starting_cards(self):
        ''' deal 8 cards to each player'''
        for n,player in self.players.items():
            for _ in range(0,8):
                self.draw_card_from_deck(player)

    def start_game(self):
        self.deck.shuffle_deck()
        self.initiate_expeditions()
        self.deal_starting_cards()

    def play_game(self):
        while len(self.deck.list) > 0:
            self.take_random_action(self.player_turn)
        self.score()
        print("end")
    
    def reset_game(self):
        pass

    def score(self):
        score_dict = {1:0, 2:0}
        for player_n, player in self.players.items():
            for colour,colour_list in player.expedition.items():
                if len(colour_list) == 0:
                    pass
                else:
                    n_wagers = 0
                    total = 0
                    for card in colour_list:
                        if card.number == 0:
                            n_wagers += 1
                        else:
                            total += card.number
                    score = (total - 20) * (n_wagers +1)
            
                    if len(colour_list) >= 8:
                        score += 20

                    print(f"player {player_n}: colour{colour}: {score} len_list = {len(colour_list) }")
                    score_dict[player_n] += score
        
        self.score_dict = score_dict

    def write_score(self):
        pass
        
    #score

In [102]:
gm = GameManager()
gm.start_game()
gm.play_game()
gm.score_dict

player 1: colourred: -24 len_list = 2
player 1: colourblue: -28 len_list = 2
player 1: colouryellow: -10 len_list = 1
player 1: colourgreen: -4 len_list = 3
player 1: colourwhite: -12 len_list = 1
player 2: colourred: -7 len_list = 2
player 2: colourblue: 18 len_list = 5
player 2: colouryellow: -11 len_list = 1
player 2: colourgreen: -6 len_list = 3
player 2: colourwhite: -1 len_list = 2
end


{1: -78, 2: -7}

{1: 17, 2: -43}

In [93]:
gm.score()

player 1: colourred: -1 len_list = 3
player 1: colourblue: -1 len_list = 3
player 1: colouryellow: 3 len_list = 5
player 1: colourgreen: 12 len_list = 5
player 1: colourwhite: 4 len_list = 4
player 2: colourred: -3 len_list = 2
player 2: colourblue: -12 len_list = 1
player 2: colouryellow: -10 len_list = 1
player 2: colourgreen: -4 len_list = 2
player 2: colourwhite: -14 len_list = 3


In [94]:
gm.player2.expedition

{'red': [<__main__.Card at 0x267582f2190>, <__main__.Card at 0x26758293370>],
 'blue': [<__main__.Card at 0x267582a7ca0>],
 'yellow': [<__main__.Card at 0x267582a7e80>],
 'green': [<__main__.Card at 0x26758307040>, <__main__.Card at 0x267583077f0>],
 'white': [<__main__.Card at 0x26758307e20>,
  <__main__.Card at 0x267583074f0>]}

In [95]:
for colour, colour_list in gm.player2.expedition.items():
    for card in colour_list:
        print(card)

red 7
red 10
blue 8
yellow 10
green 6
green 10
white 0
white 6
white 7


In [29]:
gm.take_random_action(gm.player_turn)

Player 1 taking action on card n 7, to expedition: False, draw_from: -1


In [36]:
print(gm.discard_pile["green"][0])

green 3


In [37]:
gm.player_turn

2